# Data Cleaning 
### By Swetha Revanur and Keanu Spies

Here, we implement a variety of preliminary data cleaning techniques. This notebook was adapted from [this Kaggle notebook](https://www.kaggle.com/quannguyen135/preliminary-data-cleaning-with-python). 

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta, datetime

import warnings
warnings.filterwarnings('ignore')

In [2]:
base_data_dir = '../../data'
df = pd.read_csv(os.path.join(base_data_dir, 'online_sex_work.csv'), index_col = 0)
df = df.iloc[: 28831, :]

df.head()

,Gender,Age,Location,Verification,Sexual_orientation,Sexual_polarity,Looking_for,Points_Rank,Last_login,Member_since,Number_of_Comments_in_public_forum,Time_spent_chating_H:M,Number_of_advertisments_posted,Number_of_offline_meetings_attended,Profile_pictures,Friends_ID_list,Risk
User_ID,,,,,,,,,,,,,,,,,
10386.0,male,"34,6",A,Non_Verified,Homosexual,Switch,Men,50,before_10_days,17.9.2012,32,0:2,0.0,0.0,0.0,18260,No_risk
14.0,male,"32,2",J,Non_Verified,Heterosexual,Dominant,Women,518,before_1_days,1.11.2009,710,3:45,9.0,0.0,0.0,"11778,32024,43768,23969,27318,45884,31277",No_risk
16721.0,male,"33,6",K,Non_Verified,Heterosexual,Dominant,Women,150,before_3_days,1.4.2013,25,2:15,1.0,1.0,45.0,"19805,21721,19802",No_risk
16957.0,male,34,H,Non_Verified,Heterosexual,Dominant,Women,114,before_4_days,8.4.2013,107,359:22,1.0,0.0,1.0,"40847,38183,9507,42259,5807,28118,24848,37170,...",No_risk
17125.0,male,"39,5",B,Non_Verified,Heterosexual,Dominant,Women,497,before_5_days,14.4.2013,600,0:21,0.0,6.0,8.0,"1320,35739,34231,19097,20197,18069,12330,43342...",No_risk


### Change Data Type for Some Features

In [3]:
df = df.where(df.notnull(), None)

df.index = df.index.astype(int)
df['Number_of_Comments_in_public_forum'] = df['Number_of_Comments_in_public_forum'].str.replace(' ', '').astype(int)
df['Number_of_advertisments_posted'] = df['Number_of_advertisments_posted'].astype(int)
df['Number_of_offline_meetings_attended'] = df['Number_of_offline_meetings_attended'].astype(int)
df['Profile_pictures'] = df['Profile_pictures'].astype(int)
df['Friends_ID_list'] = df['Friends_ID_list'].astype(str)
df['Risk'] = df['Risk'].astype(str)

df.head()

,Gender,Age,Location,Verification,Sexual_orientation,Sexual_polarity,Looking_for,Points_Rank,Last_login,Member_since,Number_of_Comments_in_public_forum,Time_spent_chating_H:M,Number_of_advertisments_posted,Number_of_offline_meetings_attended,Profile_pictures,Friends_ID_list,Risk
User_ID,,,,,,,,,,,,,,,,,
10386,male,"34,6",A,Non_Verified,Homosexual,Switch,Men,50,before_10_days,17.9.2012,32,0:2,0,0,0,18260,No_risk
14,male,"32,2",J,Non_Verified,Heterosexual,Dominant,Women,518,before_1_days,1.11.2009,710,3:45,9,0,0,"11778,32024,43768,23969,27318,45884,31277",No_risk
16721,male,"33,6",K,Non_Verified,Heterosexual,Dominant,Women,150,before_3_days,1.4.2013,25,2:15,1,1,45,"19805,21721,19802",No_risk
16957,male,34,H,Non_Verified,Heterosexual,Dominant,Women,114,before_4_days,8.4.2013,107,359:22,1,0,1,"40847,38183,9507,42259,5807,28118,24848,37170,...",No_risk
17125,male,"39,5",B,Non_Verified,Heterosexual,Dominant,Women,497,before_5_days,14.4.2013,600,0:21,0,6,8,"1320,35739,34231,19097,20197,18069,12330,43342...",No_risk


### Convert `Gender` to Binary Data

There are two genders present in this dataset (i.e., either explicitly male or female, or indirectly). Here, we will impute gender based on the `Sexual_orientation` and `Looking_for` fields, and then binarize the feature. `male` is `False` and `female` is `True.`

In [4]:
def fill_gender_na(row):
    if row['Sexual_orientation'] == 'Homosexual':
        if row['Looking_for'] == 'Men':
            return 'male'
        elif row['Looking_for'] == 'Women':
            return 'female'
    elif row['Sexual_orientation'] == 'Heterosexual':
        if row['Looking_for'] == 'Men':
            return 'female'
        elif row['Looking_for'] == 'Women':
            return 'male'
    else:
        return np.nan

In [5]:
fill_values = df.apply(fill_gender_na, axis = 1)
df['Gender'].fillna(fill_values, inplace = True)
df['Gender'].fillna(df['Gender'].mode()[0], inplace = True)

df.insert(0, 'Female', df['Gender'] == 'female')
del df['Gender']

df.head()

,Female,Age,Location,Verification,Sexual_orientation,Sexual_polarity,Looking_for,Points_Rank,Last_login,Member_since,Number_of_Comments_in_public_forum,Time_spent_chating_H:M,Number_of_advertisments_posted,Number_of_offline_meetings_attended,Profile_pictures,Friends_ID_list,Risk
User_ID,,,,,,,,,,,,,,,,,
10386,False,"34,6",A,Non_Verified,Homosexual,Switch,Men,50,before_10_days,17.9.2012,32,0:2,0,0,0,18260,No_risk
14,False,"32,2",J,Non_Verified,Heterosexual,Dominant,Women,518,before_1_days,1.11.2009,710,3:45,9,0,0,"11778,32024,43768,23969,27318,45884,31277",No_risk
16721,False,"33,6",K,Non_Verified,Heterosexual,Dominant,Women,150,before_3_days,1.4.2013,25,2:15,1,1,45,"19805,21721,19802",No_risk
16957,False,34,H,Non_Verified,Heterosexual,Dominant,Women,114,before_4_days,8.4.2013,107,359:22,1,0,1,"40847,38183,9507,42259,5807,28118,24848,37170,...",No_risk
17125,False,"39,5",B,Non_Verified,Heterosexual,Dominant,Women,497,before_5_days,14.4.2013,600,0:21,0,6,8,"1320,35739,34231,19097,20197,18069,12330,43342...",No_risk


### Convert European Decimal Notation in `Age`

Here, we will replace all commas (European decimal separator) with periods, while handling some unformatted values.

In [6]:
df['Age'] = df['Age'].apply(lambda x: x.replace(',', '.'))
df['Age'] = df['Age'].replace('???', np.nan)
df['Age'] = df['Age'].astype(float)
df['Age'].fillna(df['Age'].mean(), inplace = True)
df['Age'].head()

User_ID
10386    34.6
14       32.2
16721    33.6
16957    34.0
17125    39.5
Name: Age, dtype: float64

### Process `Location`

Here, we replace missing values and convert remaining location names to letters. We then generate corresponding one-hot encodings.

In [7]:
df['Location'].fillna(df['Location'].mode()[0], inplace = True)
df = df.replace({'A kraj':'P', 'Jihomoravsk_ý kraj':'Q', 'South Moravian':'Q'})
df = pd.concat([df.iloc[:, :2], pd.get_dummies(df['Location']), df.iloc[:, 3:]], axis = 1)
df.head()

,Female,Age,A,B,C,D,E,F,G,H,...,Points_Rank,Last_login,Member_since,Number_of_Comments_in_public_forum,Time_spent_chating_H:M,Number_of_advertisments_posted,Number_of_offline_meetings_attended,Profile_pictures,Friends_ID_list,Risk
User_ID,,,,,,,,,,,,,,,,,,,,,
10386,False,34.6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,50,before_10_days,17.9.2012,32,0:2,0,0,0,18260,No_risk
14,False,32.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,518,before_1_days,1.11.2009,710,3:45,9,0,0,"11778,32024,43768,23969,27318,45884,31277",No_risk
16721,False,33.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,150,before_3_days,1.4.2013,25,2:15,1,1,45,"19805,21721,19802",No_risk
16957,False,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,114,before_4_days,8.4.2013,107,359:22,1,0,1,"40847,38183,9507,42259,5807,28118,24848,37170,...",No_risk
17125,False,39.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,497,before_5_days,14.4.2013,600,0:21,0,6,8,"1320,35739,34231,19097,20197,18069,12330,43342...",No_risk


### Convert `Verification` to Binary Data

`Non_Verified` is `False` and `Verified` is `True.`

In [8]:
df['Verification'] = df['Verification'] != 'Non_Verified'
df['Verification'].head()

User_ID
10386    False
14       False
16721    False
16957    False
17125    False
Name: Verification, dtype: bool

### One-Hot Encoding for Categorical Fields (`Sexual_orientation`, `Sexual_polarity`, and `Looking_for`)

In [9]:
df = pd.concat([df.iloc[:, :22], pd.get_dummies(df['Looking_for']), df.iloc[:, 23:]], axis = 1)
df = pd.concat([df.iloc[:, :21], pd.get_dummies(df['Sexual_polarity']), df.iloc[:, 22:]], axis = 1)
df = pd.concat([df.iloc[:, :20], pd.get_dummies(df['Sexual_orientation']), df.iloc[:, 21:]], axis = 1)

df.columns

Index(['Female', 'Age', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L',
       'M', 'N', 'O', 'P', 'Q', 'S', 'Verification', 'Heterosexual',
       'Homosexual', 'bicurious', 'bisexual', 'Dominant', 'Submisive',
       'Switch', 'Men', 'Men_and_Women', 'Nobody', 'Nobody_but_maybe', 'Women',
       'Points_Rank', 'Last_login', 'Member_since',
       'Number_of_Comments_in_public_forum', 'Time_spent_chating_H:M',
       'Number_of_advertisments_posted', 'Number_of_offline_meetings_attended',
       'Profile_pictures', 'Friends_ID_list', 'Risk'],
      dtype='object')

### Process `Points_Rank`

In [10]:
df['Points_Rank'].value_counts().head(10)

0     19931
15     3995
5       427
30      333
20      275
1       248
a       209
10      195
16      169
50      167
Name: Points_Rank, dtype: int64

Aside from integers, the `Points_Rank` column contains 209 entries with `a` as their data. While it is not clear what `a` represents, 209 is relatively small compared to our dataset, so we replace all instances of `a` with the mode (0), and convert the column to integer datatype.

In [11]:
df['Points_Rank'] = df['Points_Rank'].str.replace(' ', '')
df['Points_Rank'].replace(to_replace = 'a', value = '0', inplace = True)
df['Points_Rank'] = df['Points_Rank'].astype(int)

df['Points_Rank'].head()

User_ID
10386     50
14       518
16721    150
16957    114
17125    497
Name: Points_Rank, dtype: int64

### Convert `Last_login` strings to number of days (integers)

In [12]:
df['Last_login'] = df['Last_login'].apply(lambda x: x.split('_')[1]).astype(int)
df['Last_login'].head()

User_ID
10386    10
14        1
16721     3
16957     4
17125     5
Name: Last_login, dtype: int64

### Process `Member_since`

We convert data in the `Member_since` column to Python's `datetime` datatype. An entry was evidently inputted incorrectly (it contains 0,278159722), so we replace it with the mode.

Some entries contain `dnes`, which translates "today" in Czech (probably where the dataset originates from), so we arbitrarily replace them with the maximum date among the other entries, with 1 more day added.

Finally, we also convert the date to the number of days since joining.

In [13]:
df['Member_since'].replace(to_replace = '0,278159722', value = df['Member_since'].mode()[0], inplace = True)
df['Member_since'].replace(to_replace = 'dnes', value = np.nan, inplace = True)
df['Member_since'] = pd.to_datetime(df['Member_since'], format = '%d.%m.%Y')
df['Member_since'] = df['Member_since'].fillna(df['Member_since'].max() + timedelta(days = 1))

def get_day_diff(row):
    return (datetime.now() - row).days
df['Member_since'] = df['Member_since'].apply(get_day_diff)

df['Member_since'].head()

User_ID
10386    2252
14       3303
16721    2056
16957    2049
17125    2043
Name: Member_since, dtype: int64

### Process `Time_spent_chating_H:M`

We will replace data in this column with the total number of minutes indicated. Some entries have the `H:M` (hour:minute) format, while others have what we suspect to be the `D:H:M` (day:hour:minute) format, so we define the `get_n_minutes` function to handle that.

In [14]:
def get_n_minutes(row):
    time_components = row.split(':')
    if len(time_components) == 2:
        return int(time_components[0]) * 60 + int(time_components[1])
    elif len(time_components) == 3:
        return int(time_components[0]) * 1440 + int(time_components[1]) * 60 + int(time_components[2])

df['Time_spent_chating_H:M'] = df['Time_spent_chating_H:M'].str.replace(' ', '')
df['Time_spent_chating_H:M'] = df['Time_spent_chating_H:M'].apply(get_n_minutes)

df['Time_spent_chating_H:M'].head()

User_ID
10386        2
14         225
16721      135
16957    21562
17125       21
Name: Time_spent_chating_H:M, dtype: int64

### Add `Number_of_friends` feature

We add a column to indicate the length of the `Friends_ID_List` field for each user.

In [15]:
def get_n_friends(row):
    friend_ids = row.split(',')
    if friend_ids[0] == 'None':
        return 0
    return len(friend_ids)

df.insert(40, 'Number_of_friends', df['Friends_ID_list'].apply(get_n_friends))
df.iloc[:5, 20:]

,Heterosexual,Homosexual,bicurious,bisexual,Dominant,Submisive,Switch,Men,Men_and_Women,Nobody,...,Last_login,Member_since,Number_of_Comments_in_public_forum,Time_spent_chating_H:M,Number_of_advertisments_posted,Number_of_offline_meetings_attended,Profile_pictures,Number_of_friends,Friends_ID_list,Risk
User_ID,,,,,,,,,,,,,,,,,,,,,
10386,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,10,2252,32,2,0,0,0,1,18260,No_risk
14,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1,3303,710,225,9,0,0,7,"11778,32024,43768,23969,27318,45884,31277",No_risk
16721,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3,2056,25,135,1,1,45,3,"19805,21721,19802",No_risk
16957,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4,2049,107,21562,1,0,1,12,"40847,38183,9507,42259,5807,28118,24848,37170,...",No_risk
17125,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5,2043,600,21,0,6,8,35,"1320,35739,34231,19097,20197,18069,12330,43342...",No_risk


### Process `Risk`

For each entry, we replace it with 0 if it says `No_risk`, 1 if `High_risk`, and `np.nan` if `unknown_risk`.

In [16]:
def get_risk(row):
    if row == 'No_risk':
        return 0
    elif row == 'High_risk':
        return 1
    
    return np.nan

df['Risk'] = df['Risk'].apply(get_risk)
df['Risk'].head()

User_ID
10386    0.0
14       0.0
16721    0.0
16957    0.0
17125    0.0
Name: Risk, dtype: float64

### Process Column Names

Here, we fix typos and perform casefolding on the column names.

In [17]:
df.columns = df.columns.str.lower()
df = df.rename(columns = {'submisive': 'submissive', 
                          'time_spent_chating_h:m': 'number_of_minutes_spent_chatting', 
                          'number_of_advertisments_posted': 'number_of_advertisements_posted', 
                          'profile_pictures': 'number_of_profile_pictures'})

### Save and Export

In [18]:
df = df.sort_values(by = ['risk'], na_position = 'last')
df = df.replace({'None': None})
df = df * 1 # replaces T/F with 1/0
df = df.where(df.notnull(), None)

df.to_pickle(os.path.join(base_data_dir, 'online_sex_work_clean.pkl'))